In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import glob

Uncomment and use the following Bash Script to extract individual frames @ 30 fps from videos (1..50 or as needed)

In [7]:
# %%bash

# for index in {1..50};
# do ffmpeg -i ./video_test/${index}.mp4 -r 30 -f image2 IMAGE_${index}_%d.jpg;
# done

In [10]:
# Emotion Classifier
model_fer = tf.keras.models.load_model("./fer_model_59.h5")
model_ck = tf.keras.models.load_model("./ck_model_90.h5")

# Face Classifier
classifier = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

# Labels
types_ck = ("Anger", "Disgust", "Fear", "Happy", "Sadness", "Surprise", "Contempt")
types_fer = ("Fear", "Disgust", "Anger", "Happiness", "Sadness", "Surprise", "Neutral")


for image in glob.glob('./video_test/*.jpg'):
    ret_value = cv2.imread(image)    
    
    if not ret_value:
        continue
    

    faces_detected = classifier.detectMultiScale(ret_value, 1.32, 5)

    for (x,y,w,h) in faces_detected:

        # cropping region of interest i.e. face area from image
        cv2.rectangle(ret_value, (x,y), (x+w,y+h), (255,0,0), thickness=7)
        roi=image[y:y+w,x:x+h]

        img_pixels = tf.keras.preprocessing.image.img_to_array(roi)
        img_pixels = np.expand_dims(img_pixels, axis = -1)
        img_pixels = img_pixels.reshape((-1, 48, 48, 1))

        predictions_ck = model_ck.predict(img_pixels)
        predictions_fer = model_fer.predict(img_pixels)

        # Predictions on CK dataset
        max_index_ck = np.argmax(predictions_ck[0])
        predicted_emotion_ck = types_ck[max_index_ck]

        # Predictions on FER+ Dataset
        max_index_fer = np.argmax(predictions_fer[0])
        predicted_emotion_fer = types_fer[max_index_fer]


        cv2.putText(image, predicted_emotion_ck, (int(x), int(y + 20)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, predicted_emotion_fer, (int(x), int(y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)


    cv2.imshow("Image", image)
    cv2.waitKey(1)
